In [33]:
from os import listdir
import numpy as np
import pickle

In [1]:
!mkdir clean_files
!mv files/1* clean_files/
!mv files/2* clean_files/
!mv files/3* clean_files/

In [8]:
len(listdir('clean_files'))

2415

In [29]:
def parse_hubs(raw_hubs):
    hub_list = [hub_str.strip().lower() for hub_str in raw_hubs.strip().replace(', ', '').split('\n')]
    return [hub for hub in hub_list if hub]

def get_like_value(like_str):
    if len(like_str) == 1:
        return 1 if like_str == '+' else -1
    if like_str[0] != '+':
        return -int(like_str[1:])
    return int(like_str[1:])

In [30]:
from collections import defaultdict

count_by_hub = defaultdict(int)
likes = []

In [31]:
count_good = 0

for filename in listdir('clean_files'):
    one_file = pickle.load(open(f'clean_files/{filename}', 'rb'))
    for hub in parse_hubs(one_file['hubs']):
        count_by_hub[hub] += 1
    likes.append(get_like_value(one_file['likes']))

In [35]:
print(np.mean(likes))
print(np.median(likes))

17.033954451345757
10.0


In [40]:
sorted([(count, hub) for hub, count in count_by_hub.items()], reverse=True)[:20]

[(235, 'программирование'),
 (210, 'информационная безопасность'),
 (191, 'разработка веб-сайтов'),
 (118, 'javascript'),
 (97, 'научно-популярное'),
 (93, 'машинное обучение'),
 (89, 'open source'),
 (86, 'python'),
 (84, 'карьера в it-индустрии'),
 (82, 'разработка мобильных приложений'),
 (81, 'разработка игр'),
 (75, 'it-компании'),
 (72, 'управление медиа'),
 (70, 'алгоритмы'),
 (69, 'разработка под android'),
 (67, 'it-инфраструктура'),
 (66, 'системное администрирование'),
 (64, 'управление проектами'),
 (64, 'java'),
 (64, '.net')]